---

## EVCS POI Candidate Selection

This section outlines the process of selecting POI (Point of Interest) candidates categorized by region.

- The above approach represents the process of selecting POI candidates categorized by region. POI filtering was applied when selecting POI candidates for all regions.
  
  - Before filtering, OSM_POI data was imported, and POIs were extracted for each categorized region (Atlanta, Suburban, Rural) using GIS tools.
  
  - For each extracted POI candidate, filtering was performed using the 'fclass' column, which contains information about building usage, utilizing GIS tools.

<br>

---

### Summary

- **DCFC POI Candidate Selection**: Georgia, using GIS tools & filtering.  
<br>
---


In [1]:
import yaml
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from shapely.geometry import Point, box
from shapely.ops import unary_union
from tqdm import tqdm
from rasterio.mask import geometry_mask

from src import process_ev_charging_data, greedy_optimization, setup_logging

In [ ]:
# Load the YAML file with fclass categories
yaml_file_path = 'poi_filtering.yaml'

with open(yaml_file_path, 'r') as file:
    fclass_data = yaml.safe_load(file)

# Define the region and file paths
region = 'suburban_dcfc'
gpkg_file_path = "Path to your Suburban Region POI"
output_path = "suburban_poi_candidate.gpkg"

# Load the fclass categories for candidate filtering from the YAML data
if region in fclass_data['candidate']:
    selected_fclasses = fclass_data['candidate'][region]
else:
    raise ValueError(f"Region '{region}' not found in the YAML file.")

# Read the GPKG file
poi_gdf = gpd.read_file(gpkg_file_path)

# Filter the POI data based on the selected fclass categories
selected_poi_gdf = poi_gdf[poi_gdf['fclass'].isin(selected_fclasses)]

# Get the count of filtered features
selected_feature_count = selected_poi_gdf.shape[0]
print(f"Selected feature count for {region}: {selected_feature_count}")

# Save the filtered data to a new GPKG file
selected_poi_gdf.to_file(output_path, driver='GPKG')
print(f"Filtered POI data saved for {region} at {output_path}")

## Case 4: Suburban DCFC EVCS Charger Initial Point Selection

In **Suburban areas**, high-speed chargers (DCFC) tend to be installed at intersections of major highways. To reflect this trend, the selection process considered the **road network**, **demand map**, and **POI** data. The DCFC EVCS locations in suburban areas are generally clearer compared to urban settings, and because the number of POI candidates is relatively low, running a full optimization was deemed unnecessary. Therefore, the following algorithm was used to install DCFC EVCS in all suburban areas.

### Algorithm:

1. **Selecting POIs near Motorway and Trunkway Intersections**:
   - Using **Motorway** and **Trunkway** road network data, the algorithm identifies intersections.
   - For each intersection, POIs within a **2410-meter radius** are searched.
   - Among the selected POIs, those belonging to **priority categories** (e.g., `department_store`, `hospital`, `market_place`, etc.) are prioritized.
   - If no POI is found within the priority categories, the nearest POI is randomly selected, and a **DCFC EVCS** is installed.

2. **Selecting POIs based on GeoTIFF Demand Data for Unselected Suburban Areas**:
   - For areas not covered by the intersection-based selection, the **GeoTIFF demand map** is used to ensure POIs are placed in areas with higher demand.
   - Using the `rasterio` library, the pixel value for each POI location is extracted from the demand map.
   - The algorithm first checks if a POI near a high-demand area belongs to a priority category, and if no such POI is found, it selects the nearest POI based on demand values to install the **DCFC EVCS**.

3. **Selecting POIs within Urban Polygons**:
   - The number of POIs to be selected in each **urban polygon** is determined by the variable **‘p’** associated with each polygon.
   - For each urban polygon, POIs are selected randomly, prioritizing those in **priority categories**. If the selected number of POIs is insufficient, additional POIs are selected based on **GeoTIFF demand data**.
   - If the polygon contains fewer POIs than required, additional POIs are chosen near high-demand pixels from the demand map, and the **DCFC EVCS** is installed.

By following the above steps, **DCFC EVCS** installations were successfully allocated across all suburban areas.


In [ ]:
# File paths
urban_polygon_path = 'Path to your Suburban Polygon.gpkg'
motorway_path = 'Path to your Motorways.gpkg'
trunk_path = 'Path to your Trunk.gpkg'
poi_path = 'Path to your suborban_poi_candidate.gpkg'
tif_file_path = 'Path to your Demand map'

# Parameters
radius_meters = 3000  # 3 km radius for selecting POIs near intersections
priority_categories = [
    'department_store', 'bank', 'hotel', 'theatre', 'town_hall', 'hospital', 
    'market_place', 'mall', 'public_building', 'cinema', 'vending_parking', 
    'general', 'cafe', 'fast_food', 'pharmacy', 'clothes'
]

# Load data
urban_gdf = gpd.read_file(urban_polygon_path)
motorway_gdf = gpd.read_file(motorway_path)
trunk_gdf = gpd.read_file(trunk_path)
poi_gdf = gpd.read_file(poi_path)

# Ensure correct CRS (EPSG:3857)
gdfs = [urban_gdf, motorway_gdf, trunk_gdf, poi_gdf]
for gdf in gdfs:
    if gdf.crs.to_string() != 'EPSG:3857':
        gdf.to_crs(epsg=3857, inplace=True)

# Step 1: Find intersections between motorways and trunk roads
motorway_union = motorway_gdf.unary_union
trunk_union = trunk_gdf.unary_union

intersection_points = motorway_union.intersection(trunk_union)

# Handle MultiPoint or single Point
if intersection_points.geom_type == 'MultiPoint':
    intersections_gdf = gpd.GeoDataFrame(geometry=list(intersection_points.geoms), crs='EPSG:3857')
else:
    intersections_gdf = gpd.GeoDataFrame(geometry=[intersection_points], crs='EPSG:3857')

# Step 2: Find POIs within a radius of intersection points
def get_pois_within_radius(intersections, pois, radius):
    selected_pois_list = []
    for point in intersections.geometry:
        pois['distance'] = pois.geometry.distance(point)
        selected_pois = pois[pois['distance'] <= radius]
        selected_pois_list.append(selected_pois)
    return gpd.GeoDataFrame(pd.concat(selected_pois_list, ignore_index=True), crs=pois.crs)

# Step 3: Extract nearest GeoTIFF pixel values for POIs
def assign_tif_values_to_pois(pois, tif_path):
    with rasterio.open(tif_path) as tif:
        tif_data = tif.read(1)
        transform = tif.transform

        # Assign nearest GeoTIFF values to each POI
        pois['tif_value'] = pois.geometry.apply(lambda geom: extract_pixel_value(geom, tif_data, transform))
    return pois

# Helper function to extract GeoTIFF pixel values
def extract_pixel_value(geom, tif_data, transform):
    col, row = ~transform * (geom.x, geom.y)
    col, row = int(col), int(row)
    if 0 <= row < tif_data.shape[0] and 0 <= col < tif_data.shape[1]:
        return tif_data[row, col]
    return np.nan

# Step 4: Assign POIs based on GeoTIFF values within urban polygons
def assign_pois_based_on_tif(urban_polygon, tif_path, pois, num_pois):
    with rasterio.open(tif_path) as tif:
        tif_data = tif.read(1)
        transform = tif.transform
        selected_pois = []

        for x, y in np.ndindex(tif_data.shape):
            lon, lat = transform * (x, y)
            point = Point(lon, lat)
            if urban_polygon.geometry.contains(point).any():
                pixel_value = tif_data[x, y]
                pois_near_point = pois[pois.geometry.distance(point) <= tif.res[0]]
                prioritized_pois = pois_near_point[pois_near_point['category'].isin(priority_categories)]
                
                if not prioritized_pois.empty:
                    selected_pois.append(prioritized_pois.sample(1))
                elif not pois_near_point.empty:
                    selected_pois.append(pois_near_point.sample(1))

                if len(selected_pois) >= num_pois:
                    break

        return gpd.GeoDataFrame(pd.concat(selected_pois, ignore_index=True), crs=urban_polygon.crs)

# Step 5: Select random POIs within urban polygons, using GeoTIFF for prioritization
def select_pois_within_urban(urban_gdf, pois_gdf, tif_file):
    all_selected_pois = []
    pois_gdf = assign_tif_values_to_pois(pois_gdf, tif_file)

    for idx, urban_polygon in tqdm(urban_gdf.iterrows(), total=len(urban_gdf), desc="Processing Urban Polygons"):
        pois_within_polygon = pois_gdf[pois_gdf.within(urban_polygon.geometry)]
        num_pois_to_select = int(urban_polygon['p'])  # Assuming 'p' column contains the number of POIs to select
        
        if len(pois_within_polygon) >= num_pois_to_select:
            all_selected_pois.append(pois_within_polygon.sample(num_pois_to_select))
        else:
            remaining_pois_needed = num_pois_to_select - len(pois_within_polygon)
            tif_based_pois = assign_pois_based_on_tif(urban_polygon, tif_file, pois_gdf, remaining_pois_needed)
            all_selected_pois.append(tif_based_pois)

    return gpd.GeoDataFrame(pd.concat(all_selected_pois, ignore_index=True), crs=urban_gdf.crs)

# Step 6: Perform selection of POIs near intersections
pois_near_intersections = get_pois_within_radius(intersections_gdf, poi_gdf, radius_meters)

# Step 7: Select POIs within urban polygons, considering GeoTIFF values for prioritization
selected_pois = select_pois_within_urban(urban_gdf, poi_gdf, tif_file_path)

# Save the result
selected_pois.to_file('suburban_dcfc_selection.gpkg', driver='GPKG')

print(f"Total selected POIs: {len(selected_pois)}")

## Optimization

In [2]:
# Set up logging
setup_logging()

gpkg_file = "/home/sehoon/Desktop/ACM-SIGSPATIAL-Cup-2024/data/for_model/suburban_trip_DCFC_greedy.gpkg"
tif_file = "/home/sehoon/Desktop/ACM-SIGSPATIAL-Cup-2024/data/demand_map_500.tif"
poi_file = "/home/sehoon/Desktop/ACM-SIGSPATIAL-Cup-2024/data/initial_candidate_poi/Suburban_POI_Candidate.gpkg"
output_path = "./results/suburban_dcfc/"

# Read input files
polygons = gpd.read_file(gpkg_file)
poi_gdf = gpd.read_file(poi_file)

for _, polygon in polygons.iterrows(): 
    greedy_optimization(
        polygon, 
        tif_file, 
        poi_gdf, 
        capture_range=4000, 
        bandwidth=1500, 
        constraints=(1, None), 
        output_path=output_path,
        save_intermediate=True
    )

2024-09-29 21:16:11,588 - INFO - 
*** Processed Polygon Information ***
--------------------------------------------------
Polygon ID            : Albany, GA Urban Area
Total Supply          : 11.0
A_bar Value           : 0.0006
Selected Initial Sites: 1
Initial Coverage      : 25.39%
2024-09-29 21:16:13,353 - INFO - Selecting site  2/ 2 | Selected Site:  16 | A_hat: 0.00083 | Coverage:  25.39%
2024-09-29 21:16:13,369 - INFO - Optimization process for Albany, GA Urban Area complete
2024-09-29 21:16:13,494 - INFO - 
*** Processed Polygon Information ***
-------------------------------------------------------------
Polygon ID            : Athens-Clarke County, GA Urban Area
Total Supply          : 21.0
A_bar Value           : 0.0006
Selected Initial Sites: 1
Initial Coverage      : 34.81%
2024-09-29 21:16:13,786 - INFO - Selecting site  2/ 5 | Selected Site: 177 | A_hat: 0.00080 | Coverage:  34.81%
2024-09-29 21:16:14,088 - INFO - Selecting site  3/ 5 | Selected Site: 214 | A_hat: 0.0006

KeyboardInterrupt: 